# Final Project Title


# Column Descriptions! 

YEAR Year of the Flight Trip 

MONTH Month of the Flight Trip

DAY Day of the Flight Trip 

DAY_OF_WEEK Day of week of the Flight Trip

AIRLINE Airline Identifier

FLIGHT_NUMBER Flight Identifier

TAIL_NUMBER Aircraft Identifier

ORIGIN_AIRPORT Starting Airport

DESTINATION_AIRPORT  Destination Airport

SCHEDULED_DEPARTURE  Planned Departure Time

DEPARTURE_TIMEWHEEL_OFF - TAXI_OUT

DEPARTURE_DELAY  Total Delay on Departure

TAXI_OUT The time duration elapsed between departure from the origin airport gate and wheels off

WHEELS_OFFThe time point that the aircraft's wheels leave the ground

SCHEDULED_TIMEPlanned time amount needed for the flight trip

ELAPSED_TIMEAIR_TIME+TAXI_IN+TAXI_OUT

AIR_TIMEThe time duration between wheels_off and wheels_on time

DISTANCEDistance between two airports

WHEELS_ON The time point that the aircraft's wheels touch on the ground

TAXI_IN The time duration elapsed between wheels-on and gate arrival at the destination airport

SCHEDULED_ARRIVAL Planned arrival time

ARRIVAL_TIMEWHEELS_ON+TAXI_IN

ARRIVAL_DELAY 

ARRIVAL_TIME-SCHEDULED_ARRIVAL

DIVERTED Aircraft landed on airport that out of schedule

CANCELLED Flight Cancelled (1 = cancelled)

CANCELLATION_REASON Reason for Cancellation of flight: A - Airline/Carrier; B - Weather; C - National Air System; D - Security

AIR_SYSTEM_DELAY Delay caused by air system

SECURITY_DELAY Delay caused by security

AIRLINE_DELAY Delay caused by the airline

LATE_AIRCRAFT_DELAY Delay caused by aircraft

WEATHER_DELAY Delay caused by weather

In [1]:
#read in file as dataframe 
# import pyspark modules
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark.sql.types import *       # for datatype conversion
from pyspark.sql.functions import *   # for col() function
from pyspark.mllib.linalg import DenseVector
from pyspark.ml.feature import StandardScaler
from pyspark.ml.regression import LinearRegression
import pandas as pd
import os
import pyspark.sql.types as typ
import pyspark.sql.functions as F


In [2]:
from pyspark.sql import SparkSession 
spark = SparkSession \
    .builder \
    .master("local") \
    .appName("app") \
    .config("spark.executor.memory", '2g') \
    .config('spark.executor.cores', '2') \
    .config('spark.cores.max', '2') \
    .config("spark.driver.memory",'4g') \
    .getOrCreate()

sc = SparkContext.getOrCreate()
sqlCtx = SQLContext(sc)

### Start of APT edit

In [3]:
path_to_data = os.path.join("/home/jovyan/FlightDelay/flights.csv")

read into rdd

In [4]:
delay_rdd = sc.textFile(path_to_data)

In [5]:
#delay_rdd.take(3)

read into spark dataframe

In [6]:
delay_df = spark.read.format("csv").option("header", "true").option("inferschema","true").load(path_to_data)

In [7]:
delay_df.show(3)

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

In [8]:
delay_df.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

In [9]:
delay_df.count()

5287214

In [10]:
delay_df.describe(['DEPARTURE_DELAY', 'ARRIVAL_DELAY']).show()

+-------+------------------+------------------+
|summary|   DEPARTURE_DELAY|     ARRIVAL_DELAY|
+-------+------------------+------------------+
|  count|           5208890|           5191895|
|   mean|  9.17736811489588| 4.284605717180336|
| stddev|36.605655917829274|38.808542303886945|
|    min|               -68|               -87|
|    max|              1988|              1971|
+-------+------------------+------------------+



# Check for Duplicates

In [11]:
print('rows = {}'.format(delay_df.count()))

rows = 5287214


In [12]:
print('rows = {}'.format(delay_df.distinct().count()))

rows = 5287214


Appears there are no duplicated entries

# Check for Missing Values 

In [13]:
# calculates percent of missing values in ecah column! 
missing = delay_df.agg(*[
    (1-F.count(c) / F.count('*')).alias(c + '_missing')
    for c in delay_df.columns
]).show()

+------------+-------------+-----------+-------------------+---------------+---------------------+--------------------+----------------------+---------------------------+---------------------------+----------------------+-----------------------+-------------------+-------------------+----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------------+--------------------+---------------------+--------------------+--------------------+---------------------------+------------------------+----------------------+---------------------+---------------------------+---------------------+
|YEAR_missing|MONTH_missing|DAY_missing|DAY_OF_WEEK_missing|AIRLINE_missing|FLIGHT_NUMBER_missing| TAIL_NUMBER_missing|ORIGIN_AIRPORT_missing|DESTINATION_AIRPORT_missing|SCHEDULED_DEPARTURE_missing|DEPARTURE_TIME_missing|DEPARTURE_DELAY_missing|   TAXI_OUT_missing| WHEELS_OFF_missing|SCHEDULED_TIME_missing|ELAPSED_TIME_missin

Last 6 coluns appear to have very large percentage of missing values: 

CANCELLATION_REASON_missing, AIR_SYSTEM_DELAY_missing, SECURITY_DELAY_missing, 
AIRLINE_DELAY_missing, LATE_AIRCRAFT_DELAY_missing, WEATHER_DELAY_missing 

Should I drop all of these columns? 

In [14]:
delay_df.select('WEATHER_DELAY', 'SECURITY_DELAY', 'AIR_SYSTEM_DELAY', 
                'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'CANCELLATION_REASON').show(5)

+-------------+--------------+----------------+-------------+-------------------+-------------------+
|WEATHER_DELAY|SECURITY_DELAY|AIR_SYSTEM_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|CANCELLATION_REASON|
+-------------+--------------+----------------+-------------+-------------------+-------------------+
|         null|          null|            null|         null|               null|               null|
|         null|          null|            null|         null|               null|               null|
|         null|          null|            null|         null|               null|               null|
|         null|          null|            null|         null|               null|               null|
|         null|          null|            null|         null|               null|               null|
+-------------+--------------+----------------+-------------+-------------------+-------------------+
only showing top 5 rows



# Drop columns 

In [15]:
cols_to_drop = ['WEATHER_DELAY', 'SECURITY_DELAY', 'AIR_SYSTEM_DELAY','AIRLINE_DELAY', 
                'LATE_AIRCRAFT_DELAY', 'CANCELLATION_REASON'] 

delay_df = delay_df.drop(*cols_to_drop)

In [16]:
# Drop records where atleast 3 columns have NULL values 

delay_df = delay_df.dropna(thresh=3)

Doesn't change the count of the DF, so no rows with atleast 3 missing columns 

In [17]:
delay_df.count()

5287214

# Impute Missing values 

In [ ]:
drop_cols = cols_to_exclude = ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK',
                   'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DIVERTED', 
                   'CANCELLED', 'DESTINATION_AIRPORT', 'AIRLINE', 
                   'FLIGHT_NUMBER', 'TAIL_NUMBER']


df_impute = delay_df.drop(*drop_cols)
means = df_impute.agg(*[F.mean(c).alias(c) \
                                for c in df_impute.columns]) \
                                .toPandas().to_dict('records')[0]

# Outliers 

In [ ]:
cols_to_exclude = ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK',
                   'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DIVERTED', 
                   'CANCELLED', 'DESTINATION_AIRPORT', 'AIRLINE', 
                   'FLIGHT_NUMBER', 'TAIL_NUMBER'] 

df_for_outlier_calc = delay_df.drop(*cols_to_exclude)

cols = [c for c in df_for_outlier_calc.columns]
bounds = {}

In [ ]:
for col in cols:
    quantiles = df_for_outlier_calc.approxQuantile(col, [0.25, 0.75], 0.05)
    IQR = quantiles[1] - quantiles[0]
    
    bounds[col] = [
        quantiles[0] - 1.5 * IQR,
        quantiles[1] + 1.5 * IQR
    ]

In [21]:
# append outlier indicator to data table \

outliers = df_for_outlier_calc.select(*['TAIL_NUMBER'] + [
 (
 (df_for_outlier_calc[c] < bounds[c][0]) | (df_for_outlier_calc[c] > bounds[c][1]))
    .alias(c + '_outlier') for c in cols
])

AnalysisException: "cannot resolve '`FLIGHT_NUMBER`' given input columns: [SCHEDULED_DEPARTURE, SCHEDULED_ARRIVAL, DEPARTURE_TIME, DEPARTURE_DELAY, TAXI_IN, DISTANCE, ARRIVAL_DELAY, ARRIVAL_TIME, TAXI_OUT, SCHEDULED_TIME, ELAPSED_TIME, WHEELS_ON, AIR_TIME, WHEELS_OFF];;\n'Project ['FLIGHT_NUMBER, ((cast(SCHEDULED_DEPARTURE#19 as double) < -592.5) || (cast(SCHEDULED_DEPARTURE#19 as double) > 3267.5)) AS SCHEDULED_DEPARTURE_outlier#1634, ((cast(DEPARTURE_TIME#20 as double) < 35.5) || (cast(DEPARTURE_TIME#20 as double) > 2751.5)) AS DEPARTURE_TIME_outlier#1635, ((cast(DEPARTURE_DELAY#21 as double) < -25.0) || (cast(DEPARTURE_DELAY#21 as double) > 31.0)) AS DEPARTURE_DELAY_outlier#1636, ((cast(TAXI_OUT#22 as double) < -5.5) || (cast(TAXI_OUT#22 as double) > 38.5)) AS TAXI_OUT_outlier#1637, ((cast(WHEELS_OFF#23 as double) < -78.5) || (cast(WHEELS_OFF#23 as double) > 2845.5)) AS WHEELS_OFF_outlier#1638, ((cast(SCHEDULED_TIME#24 as double) < -15.0) || (cast(SCHEDULED_TIME#24 as double) > 281.

In [ ]:
outliers.show(4)